In [ ]:
import resources.workspace as ws
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.ion();

$
% START OF MACRO DEF
% DO NOT EDIT IN INDIVIDUAL NOTEBOOKS, BUT IN macros.py
%
\newcommand{\Reals}{\mathbb{R}}
\newcommand{\Expect}[0]{\mathbb{E}}
\newcommand{\NormDist}{\mathcal{N}}
%
\newcommand{\DynMod}[0]{\mathscr{M}}
\newcommand{\ObsMod}[0]{\mathscr{H}}
%
\newcommand{\mat}[1]{{\mathbf{{#1}}}}
%\newcommand{\mat}[1]{{\pmb{\mathsf{#1}}}}
\newcommand{\bvec}[1]{{\mathbf{#1}}}
%
\newcommand{\trsign}{{\mathsf{T}}}
\newcommand{\tr}{^{\trsign}}
\newcommand{\tn}[1]{#1}
\newcommand{\ceq}[0]{\mathrel{≔}}
%
\newcommand{\I}[0]{\mat{I}}
\newcommand{\K}[0]{\mat{K}}
\newcommand{\bP}[0]{\mat{P}}
\newcommand{\bH}[0]{\mat{H}}
\newcommand{\bF}[0]{\mat{F}}
\newcommand{\R}[0]{\mat{R}}
\newcommand{\Q}[0]{\mat{Q}}
\newcommand{\B}[0]{\mat{B}}
\newcommand{\C}[0]{\mat{C}}
\newcommand{\Ri}[0]{\R^{-1}}
\newcommand{\Bi}[0]{\B^{-1}}
\newcommand{\X}[0]{\mat{X}}
\newcommand{\A}[0]{\mat{A}}
\newcommand{\Y}[0]{\mat{Y}}
\newcommand{\E}[0]{\mat{E}}
\newcommand{\U}[0]{\mat{U}}
\newcommand{\V}[0]{\mat{V}}
%
\newcommand{\x}[0]{\bvec{x}}
\newcommand{\y}[0]{\bvec{y}}
\newcommand{\z}[0]{\bvec{z}}
\newcommand{\q}[0]{\bvec{q}}
\newcommand{\br}[0]{\bvec{r}}
\newcommand{\bb}[0]{\bvec{b}}
%
\newcommand{\bx}[0]{\bvec{\bar{x}}}
\newcommand{\by}[0]{\bvec{\bar{y}}}
\newcommand{\barB}[0]{\mat{\bar{B}}}
\newcommand{\barP}[0]{\mat{\bar{P}}}
\newcommand{\barC}[0]{\mat{\bar{C}}}
\newcommand{\barK}[0]{\mat{\bar{K}}}
%
\newcommand{\D}[0]{\mat{D}}
\newcommand{\Dobs}[0]{\mat{D}_{\text{obs}}}
\newcommand{\Dmod}[0]{\mat{D}_{\text{obs}}}
%
\newcommand{\ones}[0]{\bvec{1}}
\newcommand{\AN}[0]{\big( \I_N - \ones \ones\tr / N \big)}
%
% END OF MACRO DEF
$
In tutorial [T3](T3%20-%20Univariate%20Kalman%20filtering.ipynb) we saw that the Kalman filter (KF) produces reasonable results for straight lines (in so far as linear regression does!).
What about more intricate time series?

### The model
The straight line example (of the tutorial T3) could result from discretizing the model:
\begin{align*}
\frac{d^2 x}{dt^2} &= 0 \, .
\end{align*}
Here we're going to consider the M-th order model:
$$ \frac{d^M x}{dt^M} = 0 \, .$$

This can be rewritten as a 1-st order vector (i.e. coupled system of) ODE:
$$ \begin{align}
\frac{d x_M}{dt} &= 0 \, , \\
\frac{d x_m}{dt} &= x_{m+1} \, ,
\end{align} $$
where the subscript $1,\ldots,M$ is the *index* of the state vector element.

To make it more interesting, we'll add two terms to this evolution model:  
 - damping: $\beta x_m$, with $\beta < 0$;
 - noise: $\frac{d q_m}{dt}$.  

Thus,
$$ \frac{d x_m}{dt} = \beta x_m + x_{m+1} + \frac{d q_m}{dt} \, ,$$
where $q_m$ is the noise process, and $\beta = \log(0.9)$.

Discretized by explicit-Euler, with a time step `dt=1`, this yields
$$ x_{k+1, m} = 0.9 x_{k, m} + x_{k, m+1} + q_{k, m}\, ,$$

In summary, $\x_{k+1} = \DynMod \x_k + \q_k$, with $\DynMod$ as below.

In [ ]:
M = 4 # model order (and also ndim)
M_matrix = 0.9*np.eye(M) + np.diag(np.ones(M-1), 1)
print(M_matrix)

### Estimation by the Kalman filter (and smoother) with DAPPER

Note that this is an $M$-dimensional time series.
However, we'll only observe the first (0th) component.

We shall not write the code for the multivariate Kalman filter,
because it already exists in DAPPER in `da_methods.py` and is called `ExtKF()`.

The following code configures an experiment based on the above model. Don't worry about the specifics. We'll get back to how to use DAPPER later.


In [ ]:
import dapper.mods as modelling
from dapper.mods.utils import linear_model_setup, partial_Id_Obs

# Forecast dynamics
Dyn = linear_model_setup(M_matrix, dt0=1)
Dyn['noise'] = 0.0001*(1+np.arange(M))

# Initial conditions
X0 = modelling.GaussRV(M=M, C=0.02*np.arange(M))

# Observe 0th component only
Obs = partial_Id_Obs(M, [0])
Obs['noise'] = 1000

# Time settings
t = modelling.Chronology(dt=1, dto=5, K=250)

# Wrap-up
HMM = modelling.HiddenMarkovModel(Dyn, Obs, t, X0)

This generates (simulates) a synthetic truth (xx) and observations (yy)

In [ ]:
xx, yy = HMM.simulate()
for m, x in enumerate(xx.T):
    plt.plot(x, label="x^%d"%m)
plt.legend();

Now we'll run assimilation methods on the data. Firstly, the KF, available as `ExtKF` in DAPPER:

In [ ]:
import dapper.da_methods as da
ExtKF = da.ExtKF()
ExtKF.assimilate(HMM, xx, yy)

We'll also run the "Kalman smoother" available as `ExtRTS`.
Without going into details, this method is based on the Kalman *filter* but,
being a *smoother*,
it also goes backwards and updates previous estimates with future (relatively speaking) observations.

In [ ]:
ExtRTS = da.ExtRTS()
ExtRTS.assimilate(HMM, xx, yy)

### Estimation by "time series analysis"
The following methods perform time series analysis of the observations, and are mainly derived from signal processing theory.
Considering that derivatives can be approximated by differentials, it is plausible that the above model could also be written as an AR(M) process. Thus these methods should perform quite well.

In [ ]:
# Tools
import scipy as sp
import scipy.signal as sp_sp
normalize = lambda x: x / x.sum()
truncate  = lambda x, n: np.hstack([x[:n], np.zeros(len(x)-n)])

# We only estimate the 0-th component.
signal = yy[:, 0]

# Estimated signals
ESig = {}
ESig['Gaussian'] = sp_sp.convolve(signal, normalize(sp_sp.gaussian(30, 3)), 'same')
ESig['Wiener']   = sp_sp.wiener(signal)
ESig['Butter']   = sp_sp.filtfilt(*sp_sp.butter(10, 0.12), signal, padlen=len(signal)//10)
ESig['Spline']   = sp.interpolate.UnivariateSpline(t.kko, signal, s=1e4)(t.kko)
ESig['Low-pass'] = np.fft.irfft(truncate(np.fft.rfft(signal), len(signal)//14))

### Comparison
The following code plots the results.

In [ ]:
@ws.interact(Visible=ws.SelectMultiple(options=['Truth', *ESig, 'My Method',
                                                'Kalman smoother', 'Kalman filter', 'Kalman filter a']))
def plot_results(Visible):
    plt.figure(figsize=(9, 5))
    plt.plot(t.kko, yy, 'k.', alpha=0.4, label="Obs")
    if 'Truth'           in Visible: plt.plot(t.kk , xx[:, 0]               , 'k', label="Truth")
    if 'Kalman smoother' in Visible: plt.plot(t.kk , ExtRTS.stats.mu.u[:, 0], 'm', label="K. smoother")
    if 'Kalman filter'   in Visible: plt.plot(t.kk , ExtKF .stats.mu.u[:, 0], 'b', label="K. filter")
    if 'Kalman filter a' in Visible: plt.plot(t.kko, ExtKF .stats.mu.a[:, 0], 'b', label="K. filter (a)")
    if 'My Method'       in Visible: plt.plot(t.kk , MyMeth.stats.mu.u[:, 0], 'b', label="My method")
    for method, estimate in ESig.items():
        if method in Visible: plt.plot(t.kko, estimate, label=method)

    plt.ylabel('$x^0$, $y$, and $\hat{x}^0$')
    plt.xlabel('Time index ($k$)')
    plt.legend()
    plt.show()

Visually, it's hard to imagine better performance than from the Kalman smoother.
However, recall the advantage of the Kalman filter (and smoother): *they know the forecast model that generated the truth*.

Since the noise levels Q and R are given to the DA methods (but they don't know the actual outcomes/realizations of the random noises), they also do not need any *tuning*, compared to signal processing filters, or choosing between the myriad of signal processing filters [out there](https://docs.scipy.org/doc/scipy/reference/signal.html#module-scipy.signal).

In [ ]:
def average_error(estimate_at_obs_times):
    return np.mean(np.abs(xx[t.kko, 0] - estimate_at_obs_times))

for method, estimate in {**ESig,
                         'K. smoother': ExtRTS.stats.mu.u[t.kko, 0],
                         'K. filter'  : ExtKF .stats.mu.a[:, 0],
                         # 'My Method'  : MyMeth.stats.mu.a[:, 0], # uncomment after Exc 8
                        }.items():
    print("%20s" % method, "%.4f" % average_error(estimate))

**Exc 2:** Theoretically, in the long run, the Kalman smoother should yield the optimal result. Verify this by increasing the experiment length to `K=10**4`.

**Exc 4:** Re-run the experiment with different parameters, for example the observation noise strength or `dko`.  
[Results will differ even if you changed nothing because the truth noises (and obs) are stochastic.]

**Exc 6:** Right before executing the assimilations (but after simulating the truth and obs), change $R$ by inserting:

    HMM.Obs.noise = GaussRV(C=0.01*np.eye(1))

What happens to the estimates of the Kalman filter and smoother?

**Exc 8 (optional):** Try out different methods from DAPPER by replacing `MyMethod` below with one of the following:
 - Climatology
 - Var3D
 - OptInterp
 - EnKF
 - EnKS
 - PartFilt

You typically also need to set (and possibly tune) some method parameters. Otherwise you will get an error (or possibly the method will perform very badly). You may find (some) documentation for each method in its source code...

In [ ]:
MyMeth = da.MyMethod(param1=val1, ...)
MyMeth.assimilate(HMM, xx, yy)

### Summary
Like linear regression, time series analysis is also a subset of state estimation and DA [(much of time series analysis can be formulated as state estimation)](https://www.google.com/search?q="We+now+demonstrate+how+to+put+these+models+into+state+space+form"). Moreover, DA methods produce uncertainty quantification, something which is usually more obscure with time series analysis methods. Still, the best is yet to come: DA methods should have the capacity to handle inhomogeneous, multivariate, sparsely observed, chaotic systems (which is more fun than stochastically-driven signals such as the above example).

### Next: [Dynamical systems, chaos, Lorenz](T6%20-%20Dynamical%20systems,%20chaos,%20Lorenz.ipynb)